### Inference

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!sudo apt install ffmpeg
!pip install git+https://github.com/chuangk-p/mock-aiservice.git
!pip install gradio 

  Cloning https://github.com/huggingface/transformers to c:\users\madea\appdata\local\temp\pip-req-build-dg3kz4qp
  Resolved https://github.com/huggingface/transformers to commit 74a207404e8d4524d1fdc4aa23789694f9eef347
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/402.6 kB ? eta -:--:--
   --- ------------------------------------ 30.7/402.6 kB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 194.6/402.6 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------  399.4/402.6 kB 3.1 MB/s eta 0:00:01
   -----------

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\madea\AppData\Local\Temp\pip-req-build-dg3kz4qp'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.0.0 requires transformers[sentencepiece]<4.32.0,>=4.31.0, but you have transformers 4.42.0.dev0 which is incompatible.
gradio 4.21.0 requires pydantic>=2.0, but you have pydantic 1.10.15 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/260.1 kB ? eta -:--:--
   ------------ --------------------------- 81.9/260.1 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------  256.0/260.1 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 260.1/260.1 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.6 kB ? eta -:--:--
   ---------------------------------------- 64.6/64.6 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   --------------------- ------------------ 0.6/1.0 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/184.6 kB ? eta -:--:--
   --------------------------------------- 184.6/184.6 kB 10.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'sudo' is not recognized as an internal or external command,
operable program or batch file.


  Cloning https://github.com/chuangk-p/mock-aiservice.git to c:\users\madea\appdata\local\temp\pip-req-build-bt7_zlns
  Resolved https://github.com/chuangk-p/mock-aiservice.git to commit 8ae51b4a49e630bda343e70dea73f1afc925d4ec
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for mockaiservice: filename=mockaiservice-0.1.0-py3-none-any.whl size=29476 sha256=9e564e48fae2b018272a9e866888d3737e0c5ff09f4d581aadbc37799818b032
  Stored in directory: C:\Users\madea\AppData\Local\Temp\pip-ephem-wheel-cache-770vbper\wheels\72\d6\f3\b94a6aaf5ce328e3f0475670ed57ad742764be6e4212371bc9
Successfully built mockaiservice


  Running command git clone --filter=blob:none --quiet https://github.com/chuangk-p/mock-aiservice.git 'C:\Users\madea\AppData\Local\Temp\pip-req-build-bt7_zlns'

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------- ---------------------------- 30.7/109.4 kB 1.3 MB/s eta 0:00:01
     ----------------------------------- -- 102.4/109.4 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 109.4/109.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/409.0 kB ? eta -:--:--
   ------------------- -------------------- 194.6/409.0 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 409.0/409.0 kB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   --------------- ------------------------ 0.7/1.9 MB 22.7 MB/s eta 0:00:01
   ------------------------------ --------- 1.5/1.9 MB 18.5 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.14.6
    Uninstalling pydantic_core-2.14.6:
      Successfully uninst

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightning 2.0.9.post0 requires pydantic<2.2.0,>=1.7.4, but you have pydantic 2.7.4 which is incompatible.
line-bot-sdk 3.11.0 requires requests==2.31.0, but you have requests 2.28.2 which is incompatible.
line-bot-sdk 3.11.0 requires urllib3<3,>=2.0.5, but you have urllib3 1.26.18 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from mockaiservice import setting
from mockaiservice.speech import tts
import gradio as gr

2024-06-22 18:57:17.892724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 18:57:17.892824: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 18:57:18.016205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## ASR

In [6]:
device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model="biodatlab/whisper-th-medium-combined",
    chunk_length_s=30,
    device=device,
)

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [7]:
def ASR (intput_path):
    # Perform ASR with the created pipe.
    lang = "en"
    result = pipe(intput_path, generate_kwargs={"language": lang, "task": "transcribe"}, batch_size=16)
    return result["text"]

In [7]:
# print(ASR("/kaggle/input/ai-cooking/test_asr.wav"))

/opt/conda/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 I want to know about Somdrom. Can you suggest for me?


## LLM

In [11]:
model_id = "scb10x/llama-3-typhoon-v1.5x-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def LLM (input_msg):
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant who're always speak Eng."},
        {"role": "user", "content": f"{input_msg}"},
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.4,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)
# 

In [141]:
# input_msg = "I want to travel in Thailand, can you recommend some places to visit and some local food to try? Make a 3 days trip plan for me."

# messages = [
#     {"role": "system", "content": "You are a helpful assistant who're always speak Eng."},
#     {"role": "user", "content": f"{input_msg}"},
# ]

In [142]:
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device)

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=512,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.4,
#     top_p=0.9,
# )
# response = outputs[0][input_ids.shape[-1]:]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [143]:
# print(tokenizer.decode(response, skip_special_tokens=True))

Thailand is a beautiful country with a rich culture and delicious cuisine. Here's a 3-day trip plan for you, including some must-visit places and local food to try:



Day 1:

- Morning: Start your day with a visit to the Grand Palace in Bangkok, a must-see attraction in Thailand. Make sure to wear appropriate clothing, as shorts and sleeveless shirts are not allowed.

- Afternoon: Head to Wat Phra Kaew, a temple complex located within the Grand Palace, to see the famous Emerald Buddha.

- Evening: Try some local street food at Chatuchak Weekend Market, which offers a wide variety of Thai dishes. Don't miss the chance to try Pad Thai, Tom Yum soup, and Som Tam (papaya salad).



Day 2:

- Morning: Take a day trip to Ayutthaya, a UNESCO World Heritage Site located about an hour outside of Bangkok. Visit the ancient temples and ruins, including Wat Mahathat and Wat Ratchaburana.

- Afternoon: Return to Bangkok and visit the Jim Thompson House, a museum showcasing traditional Thai archite

## TTS

In [8]:
setting.set_api_key('ejjItkAMCvhD4Hr2U39B6INZt6nO5mlh')

In [9]:
def TTS (input_msg):
    tts.convert(input_msg, './output.wav')
    return "output.wav"

In [10]:
# print(TTS("I want to travel in Thailand, can you recommend some places to visit and some local food to try? Make a 3 days trip plan for me."))

output.wav


## Gradio

In [11]:
def transcribe_and_speak(audio):
    try:
        transcription = ASR(audio) #เสียงเป็นข้อความ
        # print(transcription)
        llm_output = LLM(transcription) #ข้อความเป็นคําตอบ
        tts_output = TTS(llm_output) #คําตอบเป็นเสียง
        audio_path = "output.wav"
        with open(audio_path, "wb") as f:
            f.write(tts_output["audio"])
        return transcription, audio_path
    except Exception as e:
        print(f"Error: {e}")
        return "Error in processing", None

In [12]:
interface = gr.Interface(
    fn=transcribe_and_speak,
    inputs=gr.Audio(type="filepath"),
    outputs=["text", "audio"],
)

In [13]:
interface.launch(server_name="0.0.0.0",server_port=8000)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://846e7d985ccd76c4de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
